In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn import linear_model, tree, ensemble
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer


In [2]:
imputer = SimpleImputer(strategy = 'median')

# Sprint machine learning flow
## Name: Nguyen Anh Hoang Phuc

## Problem 1 Cross Validation
In the pre-learning period, the verification data was divided first, and the index value was calculated for that, and verification was performed. (Holdout method) However, the accuracy varies depending on the division method; in practice Cross Validation. This is a method of performing splitting multiple times and performing learning and verification for each. KFold class is provided in scikit-learn for splitting multiple times.

Create and execute code that Validation baseline model created in the pre-learning period assignment

sklearn.model_selection.KFold — scikit-learn 0.21.3 documentation

In the pre-learning period, the verification data was divided first, and the index value was calculated for that, and verification was performed. (Holdout method) However, the accuracy varies depending on the division method; in practice Cross Validation. This is a method of performing splitting multiple times and performing learning and verification for each. KFold class is provided in scikit-learn for splitting multiple times.

Create and execute code that Validation baseline model created in the pre-learning period assignment

In [3]:
### Train data

In [4]:
train_data = pd.read_csv('application_train.csv')
train_data.dropna(axis=0, subset=['TARGET'], inplace=True)
y = train_data.TARGET # Target variable 
train_data.drop(['TARGET'], axis=1, inplace=True) # Removing target variable from training data
COLUMNS = list(train_data.columns)
# get only existing data with no missing values
train_data = train_data[train_data.columns[~train_data.isnull().all()]]

### Test data

In [5]:
test_data = pd.read_csv('application_test.csv')
test_data.shape

(48744, 121)

In [6]:
COLUMNS = list(test_data.columns)
# get only existing data with no missing values
test_data = test_data[test_data.columns[~test_data.isnull().all()]]
test_data = test_data.select_dtypes('number')
test_data = test_data.fillna(test_data.median()).clip(-1e11,1e11)
test_data.shape

(48744, 105)

In [7]:
test_data.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


### Select numeric columns only


In [8]:
# Select numeric columns only
# numeric_cols = [cname for cname in train_data.columns if train_data[cname].dtypes in ['int64']]
# X = train_data[numeric_cols].copy()
train_data = train_data.select_dtypes('number')
X = train_data
X = X.fillna(X.median()).clip(-1e11,1e11)

print("Shape of input data: {} and shape of target variable: {}".format(X.shape, y.shape))

X.head() # Show first 5 training examples

Shape of input data: (307511, 105) and shape of target variable: (307511,)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Add the target back in
train_data['TARGET'] = y

In [10]:
# Lets split the data into 5 folds.  
# We will use this 'kf'(KFold splitting stratergy) object as input to cross_val_score() method
kf =KFold(n_splits=5, shuffle=True, random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 246008, Test set:61503
Fold:2, Train set: 246009, Test set:61502
Fold:3, Train set: 246009, Test set:61502
Fold:4, Train set: 246009, Test set:61502
Fold:5, Train set: 246009, Test set:61502


In [11]:
def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse= {"{:.2f}".format(rmse)}')

### Using Logistic Regression

In [12]:
score = cross_val_score(linear_model.LinearRegression(), X, y, cv= kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [-0.06977236 -0.06965837 -0.07142571 -0.06929756 -0.06988258]
rmse= 0.26


### Using Decision Tree Regression

In [13]:
score = cross_val_score(tree.DecisionTreeRegressor(random_state= 42), X, y, cv=kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [-0.15031787 -0.14885695 -0.15072681 -0.15032031 -0.14740984]
rmse= 0.39


## Problem 2 Grid search
So far, I haven't touched on the classifier parameters and used the default settings. Details of the parameters will be learned in future sprints. As a prerequisite for machine learning, it is necessary to select the optimum parameters according to the situation. Searching for the optimum parameters is called parameter tuning. A simple way to automate parameter tuning to some extent is grid search.

Please use Scikit-learn's GridSearchCV to create the code for grid search. Then do some parameter tuning on the baseline model. Please refer to the official documentation of the method used to determine which parameters to tune.

sklearn.model_selection.GridSearchCV — scikit-learn 0.21.3 documentation

To the GridSearchCV class, give the model, the search range, and the number of divisions for cross Validation as arguments. You don't need to use KFold class if you want to use it as it also includes cross Validation functionality.

In [14]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3)

In [15]:
from sklearn.model_selection import RandomizedSearchCV

#Grid
param_grid = {
    'bootstrap': [True],
    'max_depth': [3],
    'max_features': [2],
    'min_samples_leaf': [3],
    'min_samples_split': [8, 10],
    'n_estimators': [100, 200]
}

In [16]:
rf = RandomForestClassifier(max_depth=2, random_state=0)
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 2, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


GridSearchCV(cv=2,
             estimator=RandomForestClassifier(max_depth=2, random_state=0),
             n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [3],
                         'max_features': [2], 'min_samples_leaf': [3],
                         'min_samples_split': [8, 10],
                         'n_estimators': [100, 200]},
             verbose=2)

#### The Best Parameter:
I have run GridSearch on many values and here is the best parameters

In [17]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 3,
 'max_features': 2,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

## Problem 3: Survey from Kaggle Notebooks
Discover and list various ideas from Kaggle's Notebooks.

Some ideas I have found on Kaggle:
<li> Use advance models: Random Forest, Gradient Boosting
<li> Apply advance Feature Engineering: use domain knowledge features(credit_income_percent,annuity_income_percent,credit_term,days_emplyed_percent)

## Problem 4: Creating a model with high generalization performance

Please combine the idea found in Problem 3 and your own idea to create a model with high generalization performance.


As a process, please summarize in a table what you did and how much the cross Validation results changed.

### Domain Knowledge Features

In [18]:
domain_train = train_data.copy()
domain_test = test_data.copy()

In [19]:
domain_train['CREDIT_INCOME_PERCENT'] = domain_train['AMT_CREDIT'] / domain_train['AMT_INCOME_TOTAL']
domain_train['ANNUITY_INCOME_PERCENT'] = domain_train['AMT_ANNUITY'] / domain_train['AMT_INCOME_TOTAL']
domain_train['CREDIT_TERM'] = domain_train['AMT_ANNUITY'] / domain_train['AMT_CREDIT']
domain_train['DAYS_EMPLOYED_PERCENT'] = domain_train['DAYS_EMPLOYED'] / domain_train['DAYS_BIRTH']

In [20]:
domain_test['CREDIT_INCOME_PERCENT'] = domain_test['AMT_CREDIT'] / domain_test['AMT_INCOME_TOTAL']
domain_test['ANNUITY_INCOME_PERCENT'] = domain_test['AMT_ANNUITY'] / domain_test['AMT_INCOME_TOTAL']
domain_test['CREDIT_TERM'] = domain_test['AMT_ANNUITY'] / domain_test['AMT_CREDIT']
domain_test['DAYS_EMPLOYED_PERCENT'] = domain_test['DAYS_EMPLOYED'] / domain_test['DAYS_BIRTH']

### Training with Random Forest

In [21]:
rf.fit(X, y)

RandomForestClassifier(max_depth=2, random_state=0)

### Predict using Random Forest

In [22]:
rf_pred = rf.predict_proba(test_data)[:,1]
rf_pred.shape

(48744,)

In [23]:
submit = test_data[['SK_ID_CURR']]
submit['TARGET'] = rf_pred
submit.head()

ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,SK_ID_CURR,TARGET
0,100001,0.091578
1,100005,0.084732
2,100013,0.068804
3,100028,0.064782
4,100038,0.090223


In [24]:
#Write to CSV
submit.to_csv('rf_prediction.csv', index = False)

### Predict using Random Forest for Domain Knowledge Features

In [25]:
train_data.shape

(307511, 106)

In [26]:
train_domain = domain_train.drop(columns = 'TARGET')

domain_features_names = list(train_domain.columns)

# Impute the domainnomial features

domain_features = imputer.fit_transform(train_domain)
domain_features_test = imputer.transform(domain_test)

# Scale the domainnomial features
scaler = MinMaxScaler(feature_range = (0, 1))

domain_features = scaler.fit_transform(domain_features)
domain_features_test = scaler.transform(domain_features_test)

random_forest_domain = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

# Train on the training data
random_forest_domain.fit(domain_features, y)

# Extract feature importances
feature_importance_values_domain = random_forest_domain.feature_importances_
feature_importances_domain = pd.DataFrame({'feature': domain_features_names, 'importance': feature_importance_values_domain})

# Make predictions on the test data
predictions = random_forest_domain.predict_proba(domain_features_test)[:, 1]

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   28.3s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.4s finished


In [27]:
predictions.shape

(48744,)

In [28]:
domain_train

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT,CREDIT_TERM,DAYS_EMPLOYED_PERCENT
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0.0,0.0,0.0,0.0,1.0,1,2.007889,0.121978,0.060749,0.067329
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0.0,0.0,0.0,0.0,0.0,0,4.790750,0.132217,0.027598,0.070862
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0.0,0.0,0.0,0.0,0.0,0,2.000000,0.100000,0.050000,0.011814
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,NaN,NaN,NaN,NaN,NaN,0,2.316167,0.219900,0.094941,0.159905
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0.0,0.0,0.0,0.0,0.0,0,4.222222,0.179963,0.042623,0.152418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,-8456.0,...,NaN,NaN,NaN,NaN,NaN,0,1.617143,0.174971,0.108198,0.025303
307507,456252,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,-4388.0,...,NaN,NaN,NaN,NaN,NaN,0,3.743750,0.166687,0.044524,-17.580890
307508,456253,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,-6737.0,...,0.0,0.0,1.0,0.0,1.0,0,4.429176,0.195941,0.044239,0.529266
307509,456254,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,-2562.0,...,0.0,0.0,0.0,0.0,0.0,1,2.164368,0.118158,0.054592,0.400134


In [29]:
domain_train.shape

(307511, 110)

In [30]:
submit = test_data[['SK_ID_CURR']]
submit['TARGET'] = predictions
# submit
# # Save the submission dataframe
submit.to_csv('rf_domain_knowledge_prediction.csv', index = False)

ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Gradient Boosting

In [31]:
import lightgbm as lgb
import gc
from sklearn.metrics import roc_auc_score

In [32]:
def gradientboost(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics
submission, fi, metrics = gradientboost(train_data, test_data)

Training Data Shape:  (307511, 104)
Testing Data Shape:  (48744, 104)
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.793757	train's binary_logloss: 0.552943	valid's auc: 0.749536	valid's binary_logloss: 0.569006
[400]	train's auc: 0.823743	train's binary_logloss: 0.523615	valid's auc: 0.749387	valid's binary_logloss: 0.551426
Early stopping, best iteration is:
[355]	train's auc: 0.817969	train's binary_logloss: 0.529324	valid's auc: 0.749778	valid's binary_logloss: 0.554867
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.793117	train's binary_logloss: 0.553697	valid's auc: 0.752181	valid's binary_logloss: 0.567972
Early stopping, best iteration is:
[215]	train's auc: 0.795786	train's binary_logloss: 0.551059	valid's auc: 0.752289	valid's binary_logloss: 0.56642
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.792203	train's binary_logloss: 0.555149	valid's auc: 0.75679	valid's binar

In [33]:
submission

,SK_ID_CURR,TARGET
0,100001,0.268405
1,100005,0.428990
2,100013,0.123399
3,100028,0.201264
4,100038,0.566734
...,...,...
48739,456221,0.153647
48740,456222,0.296940
48741,456223,0.226013
48742,456224,0.425601


In [34]:
submission.to_csv('lbg.csv', index = False)

In [35]:
submission_domain, fi_domain, metrics_domain = gradientboost(domain_train, domain_test)

Training Data Shape:  (307511, 108)
Testing Data Shape:  (48744, 108)
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.800843	train's binary_logloss: 0.545555	valid's auc: 0.758017	valid's binary_logloss: 0.562054
Early stopping, best iteration is:
[237]	train's auc: 0.806928	train's binary_logloss: 0.539431	valid's auc: 0.758148	valid's binary_logloss: 0.558433
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.799719	train's binary_logloss: 0.546947	valid's auc: 0.761082	valid's binary_logloss: 0.561529
Early stopping, best iteration is:
[238]	train's auc: 0.806258	train's binary_logloss: 0.540422	valid's auc: 0.761158	valid's binary_logloss: 0.557712
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.799303	train's binary_logloss: 0.547742	valid's auc: 0.764886	valid's binary_logloss: 0.562918
Early stopping, best iteration is:
[203]	train's auc: 0.799834	train's binary_logloss: 0.54721

In [36]:
submission_domain
submission_domain.to_csv('lgb_domain_feature.csv', index = False)

## Problem 5: Final model selection
Ultimately, choose a model that says it's good and submit the estimated results to Kaggle to see your score. Please describe what kind of idea you adopted and what the score was

### Public Score Result:
<li> Previous Submission using Logarit Regression: 0.68426 </li>
<li> Random Forest: 0.42773 </li>
<li> Random Forest with Domain Knowledge: 0.69985 </li>
<li> Gradient Boost: 0.74453</li>
<li> Gradient Boost with Domain Knowledge: 0.76053</li>

### Private Score Result:
<li> Previous Submission using Logarit Regression: 0.68426 </li>
<li> Random Forest: 0.43378 </li>
<li> Random Forest with Domain Knowledge: 0.70319 </li>
<li> Gradient Boost: 0.74611 </li>
<li> Gradient Boost with Domain Knowledge: 0.76074</li>

For all the model and paramter i have tried, Gradient Boosting is the best one. With 0.75238 in public and 0.75294 in private. 